In [2]:
# Import modules
import ee
import geemap
import geopandas as gpd
import numpy as np

In [3]:
# Initialize Earth Engine
try:
    ee.Initialize()
except ee.EEException:
    ee.Authenticate()

In [4]:
# Open source table as a dataframe
# country_gpkg = r"data\zambia_boundary.gpkg"
country_gpkg = r"C:\Users\jedid\Documents\Code\github\crop_soil_mcda\data\zambia_boundary.gpkg"
country_gdf = gpd.read_file(country_gpkg)

# Preview the data
# country_gdf.head()

In [5]:
def get_crop_suitable_ph_image(crop, image, aoi):
    ideal_crop_ph = {
        'Soybean': [5.5, 7],
        'Groundnut': [5.5, 6.5],
        'Sugarbean': [5.5, 7],
        'Navybean': [5.5, 7],
        'Maize': [5.5, 7.5],
        'Cowpea': [5, 6.5],
        'Sunflower': [5.5, 7.5]
    }

    # Clean input
    crop = crop.strip().replace(" ","").title()

    # Mask out crop suitable ph
    crop_suitable_ph_masked = image \
        .updateMask(image \
        .gte(ideal_crop_ph[crop][0]) \
        .And(image.lte(ideal_crop_ph[crop][1]))
        )
    
    return crop_suitable_ph_masked

In [6]:
def get_vis_params(image, aoi):
    # reduceRegion to get the minimum ph in the AOI
    min_ph = image.reduceRegion(
        reducer=ee.Reducer.min(),
        geometry=aoi,
        scale=3000,  # iSDA dataset native resolution
        maxPixels=9e8
    )

    # reduceRegion to get the maximum ph in the AOI
    max_ph = image.reduceRegion(
    reducer=ee.Reducer.max(),
    geometry=aoi,
    scale=3000,  # iSDA dataset native resolution
    maxPixels=9e8
    )

    # Get the result and print the minimum value
    try:
        ph_min_max = [min_ph.getInfo()['mean_0_20'], 
                            max_ph.getInfo()['mean_0_20']]
        vis_params = {'palette': ['darkred', 'red', 
                          'white', 'lightblue', 'blue', 'darkblue'],
               'min': ph_min_max[0], 'max': ph_min_max[1]}
        
    except KeyError:
        ph_min_max = [min_ph.getInfo()['constant'], 
                            max_ph.getInfo()['constant']]
        vis_params = {'palette': ['darkred', 'yellow', 'green', 'darkgreen'],
                'min': ph_min_max[0], 'max': ph_min_max[1]}
    
    return vis_params

In [7]:
# Get weighted ph suitability
def ranked_ph_suitability_image(crop, image, aoi):
    ph_suitability_ranges = {
        'Soybean': [5.5, 6, 6.5, 7],
        'Groundnut': [5.5, 5.8, 6, 6.5],
        'Sugarbean': [5.5, 6, 6.5, 7],
        'Navybean': [5.5, 6, 6.5, 7],
        'Maize': [5.5, 6, 6.5, 7.5],
        'Cowpea': [5, 5.5, 6, 6.5],
        'Sunflower': [5.5, 6, 6.5, 7.5]
    }

    # Clean input
    crop = crop.strip().replace(" ","").title()

    # Assign suitability values based on ph ranges
    suitability_image = ee.Image(0).clip(aoi)  # Default suitability is 0 (no suitability)

    # Assign 1, low suitability
    suitability_image = suitability_image.where(image.gte(ph_suitability_ranges[crop][0]) \
                                                .And(image.lt(ph_suitability_ranges[crop][1])), 1)
    
    # Assign 2, medium suitability
    suitability_image = suitability_image.where(image.gte(ph_suitability_ranges[crop][1]) \
                                                .And(image.lt(ph_suitability_ranges[crop][2])), 2)

    # Assign 3, high suitability
    suitability_image = suitability_image.where(image.gte(ph_suitability_ranges[crop][2]), 3)
    
    return suitability_image

In [8]:
# Convert county from gdf to ee
country_ee = geemap.gdf_to_ee(country_gdf)

# Import ph dataset from GEE
raw_pH_dataset = ee.Image("ISDASOIL/Africa/v1/ph") \
            .select('mean_0_20').clip(country_ee)

crop = "Maize"

pH_dataset = raw_pH_dataset.divide(10)

# Get crop suitable average ph
suitable_ph = get_crop_suitable_ph_image(crop, pH_dataset, country_ee)

# Get ranked crop suitable average ph
ranked_suitable_ph = ranked_ph_suitability_image(crop, suitable_ph, country_ee)


In [9]:
# ph visualisastion parameters
suitable_ph_vis_params = get_vis_params(suitable_ph, country_ee)
ranked_ph_vis_params = get_vis_params(ranked_suitable_ph, country_ee)

In [13]:
# ph visualisastion parameters
vis_params = {'palette': ['darkred', 'red', 
                          'white', 'lightblue', 'blue', 'darkblue'],
                    'min': 1, 'max': 14}

# Visualise layers
m = geemap.Map()
m.zoom_to_gdf(country_gdf)

m.add_layer(country_ee, name= 'Zambia')

# m.add_ee_layer(suitable_ph, vis_params= suitable_ph_vis_params, name= f"{crop} pH")
# m.add_colorbar(vis_params, orientation='vertical',  
#                label= "pH", layer_name= f"{crop} pH Suitability")

m.add_ee_layer(ranked_suitable_ph, vis_params= ranked_ph_vis_params, name= f"{crop} pH Suitability")
m.add_colorbar(ranked_ph_vis_params, orientation='vertical',  
               label= "pH Suitability", layer_name= f"{crop} pH Suitability")
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…